#### Important user-defined variables:

In [1]:
SPECIES_NAME=Chrysophrys_auratus
SPECIES_SYMBOL=ChrAur  # Just a shortened name of your species, for naming files and genes
GENOME=/workspace/cfnjxb/PB_assembly/00_Assembly_V2/ChrAur2-renamed.scaffold.fasta
LIBRARY=/workspace/cfnjxb/PB_assembly/02_repeats/intermediate/ChrAur_scaff-families.fa
LEFT_READ=/workspace/cfnsjm/my_files/C_auratus/genomics/Samnotation/InputData/combined_snapper_trimmed_reads_left.fq.gz
RIGHT_READ=/workspace/cfnsjm/my_files/C_auratus/genomics/Samnotation/InputData/combined_snapper_trimmed_reads_right.fq.gz
LIFTOFF_REF_GFF=/workspace/cfnjxb/PB_assembly/01_annotations/intermediate_files/GCF_904848185.1_fAcaLat1.1_genomic.gff
LIFTOFF_REF_GENOME=/workspace/cfnjxb/PB_assembly/01_annotations/intermediate_files/GCF_904848185.1_fAcaLat1.1_genomic.fna
SPECIES=/workspace/cfnsjm/my_files/C_auratus/genomics/Samnotation/InputData/species.tab  # Tab separated file of species names and their orthodb name
# ODB_LINEAGE=https://bioinf.uni-greifswald.de/bioinf/partitioned_odb12/Vertebrata.fa.gz  # Currently doesn't exist. Code has been modified so this isn't needed
EMAIL=sam.modern@plantandfood.co.nz
WORKSPACE_TO_BIND_1=/workspace/cfnsjm  # This should be the workspace you're running this in

#### Variables you might want to change:

In [2]:
BASE_DIR=$PWD
ODB_VERSION_NUMBER="12"
BUSCO_LINEAGE=actinopterygii_odb10
KMER_LEN=13
LONG_TIMEOUT="4-00:00:00"
MEDIUM_TIMEOUT="2-00:00:00"
SHORT_TIMEOUT="04:00:00"
LARGE_MEM='80G'
MEDIUM_MEM='32G'
SMALL_MEM='16G'
CPUS=8
WORKSPACE_TO_BIND_2=/workspace/cfnjxb
TE_TOOLS=/workspace/cfnjxb/dfam-tetools.sh
SRC=$BASE_DIR/src
LOG=$BASE_DIR/Log
INPUT_DATA=$BASE_DIR/InputData
FINAL_OUTPUT=$BASE_DIR/FinalOutput

## A: BRAKER

In [3]:
WORKING_D=$BASE_DIR/Braker
REPMASK=$WORKING_D/01_Repmask
STAR=$WORKING_D/02_Star
BRAKER=$WORKING_D/03_Braker
BUSCO=$WORKING_D/04_Busco
LIFTOFF=$WORKING_D/05_Liftoff
MERGE_ANNOTATION=$WORKING_D/06_Merge
TMP=$WORKING_D/tmp
OUTPUT=$WORKIND_D

In [4]:
ml conda
conda deactivate
export TERM=rxvt
ml singularity
ml BUSCO
ml RepeatMasker
ml Slurm
ml STAR
export TERM=dumb

** INFO ** : singularity has been deprecated - please use apptainer in place.
Loading apptainer/1.1
  ERROR: Module cannot be loaded due to a conflict.
    HINT: Might try "module unload singularity" first.

Loading BUSCO/v5.8.2
  ERROR: Load of requirement apptainer failed


In [5]:
mkdir $FINAL_OUTPUT
mkdir $WORKING_D
mkdir $LOG
mkdir $REPMASK
mkdir $BRAKER
mkdir $STAR
mkdir $BUSCO
mkdir $LIFTOFF
mkdir $MERGE_ANNOTATION
mkdir $TMP
mkdir $TMP/singularity-tmp
mkdir $TMP/singularity-cache
export SINGULARITY_TMPDIR=$TMP/singularity-tmp
export SINGULARITY_CACHEDIR=$TMP/singularity-cache

mkdir: cannot create directory ‘/powerplant/workspace/cfnsjm/my_files/C_auratus/genomics/Samnotation/FinalOutput’: File exists
mkdir: cannot create directory ‘/powerplant/workspace/cfnsjm/my_files/C_auratus/genomics/Samnotation/Braker’: File exists
mkdir: cannot create directory ‘/powerplant/workspace/cfnsjm/my_files/C_auratus/genomics/Samnotation/Log’: File exists
mkdir: cannot create directory ‘/powerplant/workspace/cfnsjm/my_files/C_auratus/genomics/Samnotation/Braker/03_Braker’: File exists
mkdir: cannot create directory ‘/powerplant/workspace/cfnsjm/my_files/C_auratus/genomics/Samnotation/Braker/04_Busco’: File exists
mkdir: cannot create directory ‘/powerplant/workspace/cfnsjm/my_files/C_auratus/genomics/Samnotation/Braker/05_Liftoff’: File exists
mkdir: cannot create directory ‘/powerplant/workspace/cfnsjm/my_files/C_auratus/genomics/Samnotation/Braker/06_Merge’: File exists


In [6]:
sbatch --time=$SHORT_TIMEOUT --mem=$SMALL_MEM --cpus-per-task=1 --job-name=$USER" BREAKER_build" \
    --output=$LOG"/BRAKER_build-%A.out" --mail-type=ALL --mail-user=$EMAIL \
    --wrap="singularity build braker3.sif docker://teambraker/braker3:latest"
# These lines are currently commented out because the ODB partition hasn't been updated yet
# Currently the code is set up to use a pre-generated Vertebrata orthodb partition located in my workspace
#wget -P $INPUT_DATA $ODB_LINEAGE
#gzip -d $INPUT_DATA/Vertebrata.fa.gz
# Remove these lines when the proper ODB partition is updates:
vertebrata=/workspace/cfnsjm/my_files/C_auratus/genomics/OrthoDB/orthodb-clades/clades/Vertebrata.fa
ln -s $vertebrata $INPUT_DATA

In [22]:
mv braker3.sif $SRC

#### Rename sequences in the genome

In [8]:
python $SRC/rename_fasta.py $GENOME $INPUT_DATA/renamed_genome.fasta $INPUT_DATA/genome_renamed_mappings.tsv

#### 01-RepMask

In [6]:
sbatch << EOF
#!/bin/bash -e
#SBATCH --time=$MEDIUM_TIMEOUT
#SBATCH --cpus-per-task=$CPUS
#SBATCH --ntasks-per-node=1
#SBATCH --job-name=$USER" RepMask"
#SBATCH --mem=$SMALL_MEM
#SBATCH --output=$LOG"/01_Masker-%A.out"
#SBATCH --mail-type=ALL
#SBATCH --mail-user=$EMAIL

$TE_TOOLS -- RepeatMasker "$INPUT_DATA/renamed_genome.fasta" -lib "$LIBRARY" -pa $CPUS -nolow -xsmall -dir="$REPMASK"
EOF

Submitted batch job 8735832


#### 02-STAR

In [6]:
mkdir $STAR/IndexedGenome

sbatch --time=$SHORT_TIMEOUT --mem=$LARGE_MEM --cpus-per-task=$CPUS --job-name=$USER" STAR Index" --output=$LOG"/02_STAR1-%A.out" \
    --mail-type=ALL --mail-user=$EMAIL \
    --wrap="STAR --runMode genomeGenerate --runThreadN=$CPUS --genomeDir=$STAR/IndexedGenome --genomeFastaFiles=$INPUT_DATA/renamed_genome.fasta --genomeSAindexNbases=$KMER_LEN"

Submitted batch job 7767146


In [15]:
sbatch << EOF
#!/bin/bash -e
#SBATCH --time=$SHORT_TIMEOUT
#SBATCH --ntasks-per-node=1
#SBATCH --mem=$LARGE_MEM
#SBATCH --job-name=$USER" STAR"
#SBATCH --cpus-per-task=$CPUS
#SBATCH --output=$LOG"/02_STAR2-%A.out"
#SBATCH --mail-type=ALL
#SBATCH --mail-user=$EMAIL

STAR --runThreadN=$CPUS --genomeDir=$STAR/IndexedGenome \
     --readFilesIn $LEFT_READ $RIGHT_READ \
     --outFileNamePrefix $STAR/ --outSAMtype BAM Unsorted --outSAMstrandField \
     intronMotif --readFilesCommand zcat
EOF

Submitted batch job 7767151


#### 03-BRAKER

In [34]:
mkdir $BRAKER/AugustusConfig
singularity exec --bind=$WORKSPACE_TO_BIND_1,$WORKSPACE_TO_BIND_2 \
    $SRC/braker3.sif cp -a /opt/Augustus/config/. $BRAKER/AugustusConfig/

sbatch << EOF
#!/bin/bash -e
#SBATCH --time=$LONG_TIMEOUT
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=$CPUS
#SBATCH --job-name=$USER" BRAKER"
#SBATCH --mem=$LARGE_MEM
#SBATCH --output=$LOG"/03_BRAKER-%A.out"
#SBATCH --mail-type=ALL
#SBATCH --mail-user=$EMAIL

cd /home

singularity exec --bind=$WORKSPACE_TO_BIND_1,$WORKSPACE_TO_BIND_2 \
    $SRC/braker3.sif braker.pl --workingdir=$BRAKER --genome=$REPMASK/renamed_genome.fasta.masked --bam=$STAR/Aligned.out.bam  \
        --prot_seq=$INPUT_DATA/Vertebrata.fa --threads=$CPUS --species=$SPECIES_SYBMOL --AUGUSTUS_CONFIG_PATH=$BRAKER/AugustusConfig/ \
        --busco_lineage=$BUSCO_LINEAGE --gff3
        
cd -

EOF

Submitted batch job 7767272


#### 04-BUSCO

In [5]:
mkdir $BUSCO/Database
busco --download $BUSCO_LINEAGE
mv busco_downloads/* $BUSCO/Database
rm -r busco_downloads

2024-11-02 10:38:47 INFO:	Downloading information on latest versions of BUSCO data...
2024-11-02 10:38:50 INFO:	Downloading file 'https://busco-data.ezlab.org/v5/data/lineages/actinopterygii_odb10.2024-01-08.tar.gz'
2024-11-02 10:39:09 INFO:	Decompressing file '/powerplant/workspace/cfnsjm/my_files/C_auratus/genomics/Samnotation/busco_downloads/lineages/actinopterygii_odb10.tar.gz'


In [6]:
if test -f $BRAKER/bbc/better.gtf; then
  sbatch << EOF
    #!/bin/bash -e
    #SBATCH --time=$SHORT_TIMEOUT
    #SBATCH --ntasks-per-node=1
    #SBATCH --cpus-per-task=$CPUS
    #SBATCH --job-name=$USER" BUSCO"
    #SBATCH --mem=$SMALL_MEM
    #SBATCH --output=$LOG"/04_BUSCO3-%A.out"
    #SBATCH --mail-type=ALL
    #SBATCH --mail-user=$EMAIL
    
    busco --mode proteins -i $BRAKER/bbc/better.aa --out_path $BUSCO/ -o bbc -l $BUSCO_LINEAGE \
        --offline --download_path $BUSCO/Database --cpu $CPUS
EOF
fi


sbatch << EOF
#!/bin/bash -e
#SBATCH --time=$SHORT_TIMEOUT
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=$CPUS
#SBATCH --job-name=$USER" BUSCO"
#SBATCH --mem=$SMALL_MEM
#SBATCH --output=$LOG"/04_BUSCO1-%A.out"
#SBATCH --mail-type=ALL
#SBATCH --mail-user=$EMAIL

busco --mode proteins -i $BRAKER/braker.aa --out_path $BUSCO/ -o braker -l $BUSCO_LINEAGE \
    --offline --download_path $BUSCO/Database --cpu $CPUS
EOF

sbatch << EOF
#!/bin/bash -e
#SBATCH --time=$SHORT_TIMEOUT
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=$CPUS
#SBATCH --job-name=$USER" BUSCO"
#SBATCH --mem=$SMALL_MEM
#SBATCH --output=$LOG"/04_BUSCO2-%A.out"
#SBATCH --mail-type=ALL
#SBATCH --mail-user=$EMAIL

busco --mode proteins -i $BRAKER/Augustus/augustus.hints.aa --out_path $BUSCO/ -o augustus -l $BUSCO_LINEAGE \
    --offline --download_path $BUSCO/Database --cpu $CPUS
EOF

Submitted batch job 7770420
Submitted batch job 7770421


#### 05-Liftoff

In [15]:
conda activate cfnjxb_liftoff

sbatch << EOF
#!/bin/bash -e
#SBATCH -J $USER" Liftoff"	#job name
#SBATCH -o $LOG"/05_Liftoff-%A.out"	#outfile
#SBATCH --mail-user $EMAIL	#email when finished
#SBATCH --mail-type=ALL	#email settings
#SBATCH --time=$LONG_TIMEOUT	#booking time
#SBATCH -n 1	#number of processors
#SBATCH --cpus-per-task=$CPUS
#SBATCH --mem=$MEDIUM_MEM

liftoff -o $LIFTOFF/liftoff.gff -exclude_partial -polish -p $CPUS -g $LIFTOFF_REF_GFF $GENOME $LIFTOFF_REF_GENOME
EOF

conda deactivate

(/workspace/appscratch/miniconda/cfnjxb_liftoff) 
(/workspace/appscratch/miniconda/cfnjxb_liftoff) 
Submitted batch job 7768791
(/workspace/appscratch/miniconda/cfnjxb_liftoff) 
(/workspace/appscratch/miniconda/cfnjxb_liftoff) 


#### 06-MergeAnnotation

In [15]:
conda activate cfnsjm_general
braker_gtf=$BRAKER/braker.gtf
if test -f $BRAKER/bbc/better.gtf; then
    braker_gtf=$BRAKER/bbc/better.gtf
fi

sbatch --time=$SHORT_TIMEOUT --mem=$SMALL_MEM --cpus-per-task=1 --job-name=$USER" Merge" --output=$LOG"/06_Merge-%A.out" \
    --mail-type=ALL --mail-user=$EMAIL \
    --wrap="python $SRC/01_merge_annotation.py $GENOME $INPUT_DATA/genome_renamed_mappings.tsv $LIFTOFF/liftoff.gff_polished $braker_gtf $MERGE_ANNOTATION/"

conda deactivate

(/workspace/appscratch/miniconda/cfnsjm_general) 
(/workspace/appscratch/miniconda/cfnsjm_general) 
(/workspace/appscratch/miniconda/cfnsjm_general) 
(/workspace/appscratch/miniconda/cfnsjm_general) 
Submitted batch job 7771102
(/workspace/appscratch/miniconda/cfnsjm_general) 
(/workspace/appscratch/miniconda/cfnsjm_general) 


## B: OrthoFinder

In [6]:
WORKING_D=$BASE_DIR/OrthoFinder
ORTHODB_PARTITION=$INPUT_DATA/Vertebrata.fa
PROTEOME=$MERGE_ANNOTATION/merged_ok.pep
ODB_VERSION=odb${ODB_VERSION_NUMBER}v0
ORTHOFINDER=$WORKING_D/08_OrthoFinder
ADD_NAMES=$WORKING_D/09_AddGeneNames

In [7]:
conda deactivate
ml singularity
ml orthofinder

In [14]:
mkdir $WORKING_D
mkdir $INPUT_DATA/Proteomes
mkdir $ADD_NAMES
mkdir $ORTHOFINDER

mkdir: cannot create directory ‘/powerplant/workspace/cfnsjm/my_files/C_auratus/genomics/Samnotation/OrthoFinder’: File exists
mkdir: cannot create directory ‘/powerplant/workspace/cfnsjm/my_files/C_auratus/genomics/Samnotation/InputData/Proteomes’: File exists
mkdir: cannot create directory ‘/powerplant/workspace/cfnsjm/my_files/C_auratus/genomics/Samnotation/OrthoFinder/AddGeneNames’: File exists


: 1

#### 07-Download and extract data (this step can cause issues)

In [23]:
cd $INPUT_DATA
wget https://data.orthodb.org/v$ODB_VERSION_NUMBER/download/${ODB_VERSION}_gene_xrefs.tab.gz
wget https://zfin.org/downloads/uniprot.txt
wget https://data.orthodb.org/v$ODB_VERSION_NUMBER/download/${ODB_VERSION}_species.tab.gz
wget 'ftp://ftp.ncbi.nih.gov/gene/DATA/gene_info.gz'

--2024-11-03 13:48:26--  https://data.orthodb.org/v12/download/odb12v0_gene_xrefs.tab.gz
Resolving data.orthodb.org (data.orthodb.org)... 129.194.190.40
Connecting to data.orthodb.org (data.orthodb.org)|129.194.190.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4692118032 (4.4G) [application/octet-stream]
Saving to: ‘odb12v0_gene_xrefs.tab.gz’

odb12v0_gene_xrefs. 100%[===================>]   4.37G  10.9MB/s    in 6m 57s  

2024-11-03 13:55:26 (10.7 MB/s) - ‘odb12v0_gene_xrefs.tab.gz’ saved [4692118032/4692118032]

--2024-11-03 13:55:26--  https://zfin.org/downloads/uniprot.txt
Resolving zfin.org (zfin.org)... 184.171.92.30
Connecting to zfin.org (zfin.org)|184.171.92.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1937501 (1.8M) [text/plain]
Saving to: ‘uniprot.txt.1’

uniprot.txt.1       100%[===================>]   1.85M  1.91MB/s    in 1.0s    

2024-11-03 13:55:28 (1.91 MB/s) - ‘uniprot.txt.1’ saved [1937501/1937501]

--20

In [ ]:
gzip -d ${ODB_VERSION}_gene_xrefs.tab.gz
gzip -d ${ODB_VERSION}_species.tab.gz
mv $SPECIES species.tab
gzip -d gene_info.gz
mv gene_info gene_info.tab
cut -f 2 species.tab > taxa_ids.txt
sed -i -e 's/^/\^/' taxa_ids.txt
srun --mem='16G' --time='01:00:00' grep -f taxa_ids.txt ${ODB_VERSION}_gene_xrefs.tab > gene_xrefs.tab
grep 'NCBIgid$' gene_xrefs.tab > gene_ncbi_ids.tab
cut -d '_' -f 1 taxa_ids.txt > taxa_ids.tab
mv taxa_ids.tab taxa_ids.txt
sed -i 's/$/\t/' taxa_ids.txt
grep -f taxa_ids.txt gene_info.tab | awk -F'\t' '{print $1"\t"$2"\t"$3"\t"$9"\t"$10}' > subset_gene_info.tab
sort -k2,2 subset_gene_info.tab > subset_gene_info_sorted.tab
sort -k2,2 gene_ncbi_ids.tab > gene_ncbi_ids_sorted.tab
head -n 1 gene_info.tab | awk -F'\t' '{print $1"\t"$2"\t"$3"\t"$9"\t"$10"\todb_gene\tignore"}' > joined_gene_info.tab
join -1 2 -2 2 -t $'\t' subset_gene_info_sorted.tab gene_ncbi_ids_sorted.tab >> joined_gene_info.tab
rm gene_info.tab
rm taxa_ids.txt
rm ${ODB_VERSION}_gene_xrefs.tab
cd -


gzip: odb12v0_gene_xrefs.tab.gz: decompression OK, trailing garbage ignored
mv: '/workspace/cfnsjm/my_files/C_auratus/genomics/Samnotation/InputData/species.tab' and 'species.tab' are the same file
srun: job 7771104 queued and waiting for resources
srun: job 7771104 has been allocated resources


In [20]:
cp $PROTEOME $INPUT_DATA/Proteomes/$SPECIES_NAME.pep
sbatch --time=$SHORT_TIMEOUT --mem=$MEDIUM_MEM --job-name=$USER" 07_ExtractProteomes" \
    --mail-type=ALL --mail-user=$EMAIL --output=$LOG"/07_Proteomes-%A.out" \
    --wrap="python $SRC/extract_proteomes.py $ORTHODB_PARTITION $INPUT_DATA/species.tab $INPUT_DATA/Proteomes"

Submitted batch job 7772077


#### 08-OrthoFinder

In [28]:
sbatch --time=$MEDIUM_TIMEOUT --mem=$MEDIUM_MEM --cpus-per-task=$CPUS --job-name=$USER" 08_OrthoFinder" \
    --output=$LOG"/08_OrthoFinder-%A.out" --mail-type=ALL --mail-user=$EMAIL\
    --wrap="orthofinder -t $CPUS -f $INPUT_DATA/Proteomes/ -o $ORTHOFINDER/Results"

Submitted batch job 7772151


In [34]:
mv $ORTHOFINDER/Results/* $FINAL_OUTPUT/OrthoFinder

#### 09-Add gene names

In [8]:
conda activate cfnsjm_general

sbatch --time=$MEDIUM_TIMEOUT --mem=$SMALL_MEM --cpus-per-task=1 --job-name=$USER" Names" --output=$LOG"/09_Names-%A.out" \
    --mail-type=ALL --mail-user=$EMAIL \
    --wrap="python $SRC/temp.py $MERGE_ANNOTATION/merged.gff $INPUT_DATA/joined_gene_info.tab $FINAL_OUTPUT/OrthoFinder/ \
    $ADD_NAMES/ ${SPECIES_SYMBOL}.gff $SPECIES_SYMBOL"

conda deactivate

(/workspace/appscratch/miniconda/cfnsjm_general) 
(/workspace/appscratch/miniconda/cfnsjm_general) 
Submitted batch job 8486276
(/workspace/appscratch/miniconda/cfnsjm_general) 
(/workspace/appscratch/miniconda/cfnsjm_general) 


## Finalise!

In [7]:
mv $ADD_NAMES/${SPECIES_SYMBOL}.gff $FINAL_OUTPUT
mv $ADD_NAMES/orthogroups_with_names.tsv $FINAL_OUTPUT

In [8]:
rm -r $REPMASK
rm -r $STAR
rm -r $TMP
rm -r $ORTHOFINDER
rm -r $BUSCO/Database
rmdir $ADD_NAMES
rmdir $WORKING_D

## Utility Blocks:

In [1]:
sacct --format=jobid,jobname,partition,state,TimeLimit,exitcode,start,end,maxvmsize,alloccpus,ReqMem,allocnodes,ntasks -j 8571905

JobID           JobName  Partition      State  Timelimit ExitCode               Start                 End  MaxVMSize  AllocCPUS     ReqMem AllocNodes   NTasks 
------------ ---------- ---------- ---------- ---------- -------- ------------------- ------------------- ---------- ---------- ---------- ---------- -------- 
8571905      Kraken2_c+      short OUT_OF_ME+   20:00:00    0:125 2025-05-09T14:40:28 2025-05-09T14:44:56                    16        64G          1          
8571905.bat+      batch            OUT_OF_ME+               0:125 2025-05-09T14:40:28 2025-05-09T14:44:56          0         16                     1        1 
8571905.ext+     extern             COMPLETED                 0:0 2025-05-09T14:40:28 2025-05-09T14:44:56          0         16                     1        1 


In [20]:
echo $GENOME
echo $INPUT_DATA/genome_renamed_mappings.tsv 
echo $LIFTOFF/liftoff.gff_polished 
echo $BRAKER/braker.gtf

/workspace/cfnjxb/PB_assembly/00_Assembly_V2/ChrAur2-renamed.scaffold.fasta
/powerplant/workspace/cfnsjm/my_files/C_auratus/genomics/Samnotation/InputData/genome_renamed_mappings.tsv
/powerplant/workspace/cfnsjm/my_files/C_auratus/genomics/Samnotation/Braker/05_Liftoff/liftoff.gff_polished
/powerplant/workspace/cfnsjm/my_files/C_auratus/genomics/Samnotation/Braker/03_Braker/braker.gtf
